In [3]:
import matplotlib.pyplot as plt 
import numpy as np
from cerebellum_functionnal import *
import pprint as pp

In [4]:
def mean_weight(coupling,nb_connect,size_a,size_b):
    return coupling*(1/((nb_connect*size_a)/size_b))

def minimum_connect(nb_connect,size_a,size_b):
    return int(((nb_connect*size_a)/size_b)/2)

In [5]:
input_size=100
DCN_size=100
IO_size=100
GC_size=100
PC_size=1000
coupling_GC_PC=1
coupling_PC_DCN=5
coupling_DCN_IO=5
coupling_IO_PC=3
coupling_PT_GC=1
coupling_PT_DCN=1
coupling_PT_IO=1
# The number of connection a neuron from the source population will make with the target population
nb_connect_GC_PC=100 # WARNING, certainly to wide for a normal activity vector
nb_connect_PC_DCN=2
nb_connect_DCN_IO=10
nb_connect_IO_PC=None# the relative sizes of populations are enough to determine the number of connections
nb_connect_PT_GC=10
nb_connect_PT_DCN=10
nb_connect_PT_IO=10
# The minimum of connection a neuron from the target population should get from the source population
minimum_connect_GC_PC = minimum_connect(nb_connect_GC_PC,GC_size,PC_size)
minimum_connect_PC_DCN = minimum_connect(nb_connect_PC_DCN,PC_size,DCN_size)
minimum_connect_DCN_IO = minimum_connect(nb_connect_DCN_IO,DCN_size,IO_size)
minimum_connect_IO_PC = None # as the number of connection is determinist base on the relative sizes of populations 
minimum_connect_PT_GC = minimum_connect(nb_connect_PT_GC,input_size,GC_size)
minimum_connect_PT_DCN = minimum_connect(nb_connect_PT_DCN,input_size,DCN_size)
minimum_connect_PT_IO = minimum_connect(nb_connect_PT_IO,input_size,IO_size)


cere_info = dict()
cere_info["neuron_pop"]=dict()
cere_info["neuron_pop"]["DCN"]=dict()
cere_info["neuron_pop"]["DCN"]["size"]=DCN_size

cere_info["neuron_pop"]["IO"]=dict()
cere_info["neuron_pop"]["IO"]["size"]=IO_size

cere_info["neuron_pop"]["GC"]=dict()
cere_info["neuron_pop"]["GC"]["size"]=GC_size

cere_info["neuron_pop"]["PC"]=dict()
cere_info["neuron_pop"]["PC"]["size"]=PC_size

cere_info["neuron_pop"]["PT"]=dict()
cere_info["neuron_pop"]["PT"]["size"]=input_size

# create an activity vector variable for each region of the cerebellum (any local variable is considered a vector
# and a tuple is used to store the shape)
for key in cere_info["neuron_pop"]:
    cere_info["neuron_pop"][key]["local_variables"]=dict()
    cere_info["neuron_pop"][key]["local_variables"]["activity"] = (cere_info["neuron_pop"][key]["size"],1)
    pass

# Purkinje cells are a little bit special as we add a local variable called calcium to deal with synaptic plasticity

cere_info["neuron_pop"]["PC"]["local_variables"]["calcium"] = (cere_info["neuron_pop"]["PC"]["size"],1)

""" probability of connection based on the populations sizes 
and the divergence factor """

cere_info["synapse_pop"]=dict()
cere_info["synapse_pop"]["GC_PC"]=dict()
cere_info["synapse_pop"]["GC_PC"]["i_o"]=("GC","PC")
cere_info["synapse_pop"]["GC_PC"]["type"]="excitatory"
cere_info["synapse_pop"]["GC_PC"]["nb_connect"]=nb_connect_GC_PC
cere_info["synapse_pop"]["GC_PC"]["minimum_connect"]=minimum_connect_GC_PC
cere_info["synapse_pop"]["GC_PC"]["plastic"] = True

cere_info["synapse_pop"]["PC_DCN"]=dict()
cere_info["synapse_pop"]["PC_DCN"]["i_o"]=("PC","DCN")
cere_info["synapse_pop"]["PC_DCN"]["type"]="inhibitory"
cere_info["synapse_pop"]["PC_DCN"]["nb_connect"]=nb_connect_PC_DCN
cere_info["synapse_pop"]["PC_DCN"]["minimum_connect"]=minimum_connect_PC_DCN
cere_info["synapse_pop"]["PC_DCN"]["plastic"] = False

cere_info["synapse_pop"]["DCN_IO"]=dict()
cere_info["synapse_pop"]["DCN_IO"]["i_o"]=("DCN","IO")
cere_info["synapse_pop"]["DCN_IO"]["type"]="inhibitory"
cere_info["synapse_pop"]["DCN_IO"]["nb_connect"]=nb_connect_DCN_IO
cere_info["synapse_pop"]["DCN_IO"]["minimum_connect"]=minimum_connect_DCN_IO
cere_info["synapse_pop"]["DCN_IO"]["plastic"] = False

cere_info["synapse_pop"]["IO_PC"]=dict()
cere_info["synapse_pop"]["IO_PC"]["i_o"]=("IO","PC")
cere_info["synapse_pop"]["IO_PC"]["type"]="Undefined"
cere_info["synapse_pop"]["IO_PC"]["nb_connect"]= nb_connect_IO_PC
cere_info["synapse_pop"]["IO_PC"]["minimum_connect"]= minimum_connect_IO_PC
cere_info["synapse_pop"]["IO_PC"]["plastic"] = False

# cere_info["synapse_pop"]["PT_GC"]=dict()
# cere_info["synapse_pop"]["PT_GC"]["i_o"]=("PT","GC")
# cere_info["synapse_pop"]["PT_GC"]["type"]="excitatory"  #PT->GC DISABLED !
# cere_info["synapse_pop"]["PT_GC"]["nb_connect"]=nb_connect_PT_GC
# cere_info["synapse_pop"]["PT_GC"]["minimum_connect"]= minimum_connect_PT_GC
# cere_info["synapse_pop"]["PT_GC"]["plastic"] = False

cere_info["synapse_pop"]["PT_DCN"]=dict()
cere_info["synapse_pop"]["PT_DCN"]["i_o"]=("PT","DCN")
cere_info["synapse_pop"]["PT_DCN"]["type"]="excitatory"
cere_info["synapse_pop"]["PT_DCN"]["nb_connect"]=nb_connect_PT_DCN
cere_info["synapse_pop"]["PT_DCN"]["minimum_connect"]= minimum_connect_PT_DCN
cere_info["synapse_pop"]["PT_DCN"]["plastic"] = False

# cere_info["synapse_pop"]["PT_IO"]=dict()
# cere_info["synapse_pop"]["PT_IO"]["i_o"]=("PT","IO")
# cere_info["synapse_pop"]["PT_IO"]["type"]="excitatory" #PT->IO DISABLED !
# cere_info["synapse_pop"]["PT_IO"]["nb_connect"]=nb_connect_PT_IO
# cere_info["synapse_pop"]["PT_IO"]["minimum_connect"]= minimum_connect_PT_IO
# cere_info["synapse_pop"]["PT_IO"]["plastic"] = False

cere_info["synapse_pop"]["GC_PC"]["mean_weight"]=mean_weight(coupling_GC_PC,nb_connect_GC_PC,GC_size,PC_size) 
cere_info["synapse_pop"]["PC_DCN"]["mean_weight"]=mean_weight(coupling_PC_DCN,nb_connect_PC_DCN,PC_size,DCN_size) 
cere_info["synapse_pop"]["DCN_IO"]["mean_weight"]=mean_weight(coupling_DCN_IO,nb_connect_DCN_IO,DCN_size,IO_size) 
cere_info["synapse_pop"]["IO_PC"]["mean_weight"]=coupling_IO_PC # as we have a one to one connectivity
#cere_info["synapse_pop"]["PT_GC"]["mean_weight"]=mean_weight(coupling_PT_GC,nb_connect_PT_GC,input_size,GC_size) 
cere_info["synapse_pop"]["PT_DCN"]["mean_weight"]=mean_weight(coupling_PT_DCN,nb_connect_PT_DCN,input_size,DCN_size) 
#cere_info["synapse_pop"]["PT_IO"]["mean_weight"]=mean_weight(coupling,nb_connect_PT_IO,input_size,IO_size) 

for key in cere_info["synapse_pop"]:
    cere_info["synapse_pop"][key]["local_variables"]=dict()
    input_pop = cere_info["synapse_pop"][key]["i_o"][0]
    input_pop_size = cere_info["neuron_pop"][input_pop]["size"]
    target_pop = cere_info["synapse_pop"][key]["i_o"][1]
    target_pop_size = cere_info["neuron_pop"][target_pop]["size"]
    cere_info["synapse_pop"][key]["local_variables"]["weight"] = (input_pop_size,target_pop_size)

In [6]:
def connected_to(ind_source,source,target):
    return set(np.where(cere_state["synapse_pop"][source+"_"+target]["local_variables"]["connectivity"][ind_source]==1)[0])
    
def connected_to_pop(ind_sources,source,target):
    connected=set()
    for ind_source in ind_sources:
        connected= connected.union(connected_to(ind_source,source,target))
    return connected

In [7]:
nb_time_steps=1000

In [8]:
"Generate dict input :"

dict_input=dict()
for neuron_pop in cere_info["neuron_pop"]:
    dict_input[neuron_pop]=dict()
    for local_variable in cere_info["neuron_pop"][neuron_pop]["local_variables"]:
        dict_input[neuron_pop][local_variable]= np.zeros((nb_time_steps,cere_info["neuron_pop"][neuron_pop]["size"]))

stimulated_PT=[0,1,2,3,4,5]
stimulated_GC=[0,1,2,3,4,5]
stimulated_IO=[0,1,2,3,4,5]
dict_input["PT"]["activity"][:,stimulated_PT] =1 
dict_input["GC"]["activity"][:,stimulated_GC] =1 
dict_input["IO"]["activity"][:,stimulated_IO] =1 

In [9]:
t = 0 
activity_t = dict()
for cere_state in whole_model_sim(cere_info,dict_input):
    if t==0:
        for neuron_pop in cere_state["neuron_pop"]:
            activity_t[neuron_pop]=np.zeros((nb_time_steps,cere_info["neuron_pop"][neuron_pop]["size"]))
        activity_t["GC_PC"]=np.zeros((nb_time_steps,cere_info["neuron_pop"]["GC"]["size"],cere_info["neuron_pop"]["PC"]["size"]))
    else :
        for neuron_pop in cere_state["neuron_pop"]:
            activity_t[neuron_pop][t,:]=cere_state["neuron_pop"][neuron_pop]["local_variables"]["activity"]
        activity_t["GC_PC"][t,:]=cere_state["synapse_pop"]["GC_PC"]["local_variables"]["weight"]
    if t==nb_time_steps-1:
        cere_state_final=cere_state
        break
    t+=1

In [184]:
#plt.plot([np.mean(i.flatten()) for i in activity_t["GC_PC"]])#we need to remove all zeros
#plt.plot([np.mean(t) for t in activity_t["PC"]])
#plt.hist(activity_t["DCN"][-1]);

In [185]:
connected_io_pc_zero = connected_to_pop(stimulated_IO,"IO","PC")
connected_gc_pc_zero = connected_to_pop(stimulated_GC,"GC","PC")
connected_pt_dcn_zero = connected_to_pop(stimulated_PT,"PT","DCN")
connected_pc_dcn_pop = connected_to_pop(connected_gc_pc_zero,"PC","DCN")
over_pc = connected_gc_pc_zero.intersection(connected_io_pc_zero)
connected_over_pc_dcn_pop = connected_to_pop(over_pc,"PC","DCN")
reach_io = connected_to_pop(connected_over_pc_dcn_pop,"DCN","IO")


In [187]:
len(reach_io)

100

In [126]:
connected_in_time_mean = list()
connected_in_time = list()

for t in range(1,nb_time_steps):
    connected_in_time_mean.append(np.mean(activity_t["GC_PC"][t][:,connected_gc_pc_zero].flatten()[connected_and_on]))
    connected_in_time.append(activity_t["GC_PC"][t][:,connected_pc].flatten()[connected_and_on])

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# reach_io_len_mean=0
# sample = 100
# for it in range(sample):
#     t = 0 
#     activity_t = dict()
#     for cere_state in whole_model_sim(cere_info,dict_input):
#         if t==0:
#             for neuron_pop in cere_state["neuron_pop"]:
#                 activity_t[neuron_pop]=np.zeros((nb_time_steps,cere_info["neuron_pop"][neuron_pop]["size"]))
#             activity_t["GC_PC"]=np.zeros((nb_time_steps,cere_info["neuron_pop"]["GC"]["size"],cere_info["neuron_pop"]["PC"]["size"]))
#         else :
#             for neuron_pop in cere_state["neuron_pop"]:
#                 activity_t[neuron_pop][t,:]=cere_state["neuron_pop"][neuron_pop]["local_variables"]["activity"]
#             activity_t["GC_PC"][t,:]=cere_state["synapse_pop"]["GC_PC"]["local_variables"]["weight"]
#         if t==nb_time_steps-1:
#             cere_state_final=cere_state
#             break
#         t+=1

#     connected_io_pc_zero = connected_to_pop(stimulated_IO,"IO","PC")
#     connected_gc_pc_zero = connected_to_pop(stimulated_GC,"GC","PC")
#     connected_pt_dcn_zero = connected_to_pop(stimulated_PT,"PT","DCN")
#     connected_pc_dcn_pop = connected_to_pop(connected_gc_pc_zero,"PC","DCN")
#     over_pc = connected_gc_pc_zero.intersection(connected_io_pc_zero)
#     #over_dcn = connected_pc_dcn_pop.intersection(connected_pt_dcn_zero)
#     connected_over_pc_dcn_pop = connected_to_pop(over_pc,"PC","DCN")
#     reach_io = connected_to_pop(connected_over_pc_dcn_pop,"DCN","IO")
#     reach_io_len_mean+=len(reach_io)